In [1]:
            # !pip install alpaca_trade_api

     |████████████████████████████████| 40 kB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 74 kB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 88 kB 6.8 MB/s  eta 0:00:01


In [2]:
import alpaca_trade_api as api

In [ ]:
api = tradeapi.REST()


In [41]:
import os
files = glob.glob('/Users/evienguyen/Documents/FinRL/FinRL/finrl/results/actions_trade_ensemble*')
files.sort(key=os.path.getmtime)

action_trade =  sorted(files,key=os.path.getmtime)[-1]

/Users/evienguyen/Documents/FinRL/FinRL/finrl/results/actions_trade_ensemble_441.csv
/Users/evienguyen/Documents/FinRL/FinRL/finrl/results/actions_trade_ensemble_504.csv
/Users/evienguyen/Documents/FinRL/FinRL/finrl/results/actions_trade_ensemble_567.csv
/Users/evienguyen/Documents/FinRL/FinRL/finrl/results/actions_trade_ensemble_630.csv
/Users/evienguyen/Documents/FinRL/FinRL/finrl/results/actions_trade_ensemble_693.csv
/Users/evienguyen/Documents/FinRL/FinRL/finrl/results/actions_trade_ensemble_756.csv
/Users/evienguyen/Documents/FinRL/FinRL/finrl/results/actions_trade_ensemble_819.csv
/Users/evienguyen/Documents/FinRL/FinRL/finrl/results/actions_trade_ensemble_882.csv
/Users/evienguyen/Documents/FinRL/FinRL/finrl/results/actions_trade_ensemble_945.csv
/Users/evienguyen/Documents/FinRL/FinRL/finrl/results/actions_trade_ensemble_1008.csv
/Users/evienguyen/Documents/FinRL/FinRL/finrl/results/actions_trade_ensemble_1071.csv
/Users/evienguyen/Documents/FinRL/FinRL/finrl/results/actions_t

In [10]:
import pandas as pd

actions_trade = pd.read_csv('/Users/evienguyen/Documents/FinRL/FinRL/results/actions_trade_ensemble_1260.csv')

In [34]:
for ticker in actions_trade.columns:
    if actions_trade.iloc[-1][ticker] > 0:
        api.submit_order(
            symbol=ticker,
            qty=actions_trade.iloc[-1][ticker],
            side='buy',
            type='market',
            time_in_force='gtc'
        )
    elif actions_trade.iloc[-1][ticker] < 0:
        api.submit_order(
            symbol=ticker,
            qty= actions_trade.iloc[-1][ticker],
            side='sell',
            type='market',
            time_in_force='gtc'
        )

100